In [ ]:
import numpy as np
import pandas as pd

# Hàm text để xử lý văn bản
from sklearn.feature_extraction import text

# Hàm CountVectorizer để đếm số lần xuất hiện của từ trong văn bản và biến nó thành ma trận
from sklearn.feature_extraction.text import CountVectorizer

# Hàm cosine_similatity để tính toán độ tương đồng cosine giữa các vector
from sklearn.metrics.pairwise import cosine_similarity

import re
import string

In [ ]:
# Đọc file henime2024.csv
anime = pd.read_csv("D:\Visual studio code\CheckertheAnime\henime2024.csv")

# Xuất ra 5 hàng đầu của bảng
anime.head()

---

MAL_ID (int): Mã số của bộ phim trên trang web “My Anime List”

Name (text): Tên của phim

Score (float): Số điểm trung bình của phim trên thang điểm 10

Genres (text): Thể loại của bộ phim (Hài, Hành động, Lãng mạn, Kinh dị, …)

English name (text): Tên bộ phim theo tiếng Anh 

Japanese name (text): Tên bộ phim theo tiếng Nhật

Type (text): Hình thức công chiếu của phim (Chiếu TV, chiếu rạp, phần phim mở rộng,…)

Episodes (int): Số tập của phim

Aired (date/time): Thời gian công chiếu phim

Premiered (text): Mùa công chiếu phim (Mùa hè năm 1990, Mùa đông năm 2006,…)

Producers (text): Nhà sản xuất

Licensors (text): Đơn vị cấp phép công chiếu

Studios (text): Hãng phim

Source (text): Nguồn gốc của phim (Truyện tranh, tiểu thuyết, game,…)

Duration (text): Độ dài mỗi tập

Rating (text): Xếp hạng độ tuổi

Ranked (int): Xếp hạng của phim

Popularity (int): Độ phổ biến của phim, xếp hạng dựa vào số người dùng thêm vào danh sách xem của họ

Members (int): Số lượng thành viên cộng đồng nằm trong “nhóm” của bộ phim này

Favorites (int): Số người thêm bộ phim vào danh sách yêu thích

Watching (int): Số người đang xem phim

Completed (int): Só người đã xem xong bộ phim

On-Hold (int): Số người đang tạm dừng xem

Dropped (int): Số người đã bỏ xem phim

Plan to Watch (int): Số người lên kế hoạch xem phim

Score-10 (int): Số người đánh giá 10 cho phim

Score-9 (int): Số người đánh giá 9 cho phim

Score-8 (int): Số người đánh giá 8 cho phim

Score-7 (int): Số người đánh giá 7 cho phim

Score-6 (int): Số người đánh giá 6 cho phim

Score-5 (int): Số người đánh giá 5 cho phim

Score-4 (int): Số người đánh giá 4 cho phim

Score-3 (int): Số người đánh giá 3 cho phim

Score-2 (int): Số người đánh giá 2 cho phim

Score-1 (int): Số người đánh giá 1 cho phim

---

Ta sẽ nhập tên hoặc thể loại một bộ phim. Mô hình sau đó sẽ xuất ra tên của một số bộ phim cùng thể loại với phim vừa nhập.

Kiểu dữ liệu của thuộc tính cần dự đoán: Gợi ý theo bản ghi

In [ ]:
# Giữ lại các cột Name, Genres, Type, Episodes
anime = anime[['Name', 'Genres', 'Type', 'Episodes']]

anime.head()

In [ ]:
# Định nghĩa hàm clean() để sửa văn bản như chuyển chữ hoa thành chữ thường, loại bỏ các dấu câu, loại bỏ URL, các thẻ HTML,...
def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text) #loại bỏ chữ hoa
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

# Áp dụng hàm clean() cho cột Name để tạo thành cột mới là cleaned_Name
anime["cleaned_Name"] = anime["Name"].apply(clean)

# Áp dụng hàm clean() cho cột Genres để tạo thành cột mới là cleaned_Genres
anime["cleaned_Genres"] = anime["Genres"].apply(clean)

# Kết hợp 2 cột trên thành một cột mới có tên cleaned_NameGenres
anime["cleaned_NameGenres"] = anime["cleaned_Name"] + " " + anime["cleaned_Genres"]

anime.head()

In [ ]:
# Tạo một Series có tên "indices" có công dụng để tìm kiếm nhanh dựa trên cleaned_NameGenres và loại bỏ các thành phần trùng lặp drop_duplicates()
indices = pd.Series(anime.index,index=anime['cleaned_NameGenres']).drop_duplicates()

In [ ]:
# Định nghĩa một hàm get_similarity() để tính độ tương đồng giữa tên anime nhập vào và danh sách tên phim trong bảng
def get_similarity(title):
    
    # Khởi tạo một biến có tên "new_title" có giá trị = False để kiểm tra tên anime nhập vào có tồn tại trong bảng hay không
    new_title = False
    
    # Tạo danh sách "feature" bằng cách lấy cột "cleaned_NameGenres" và chuyển thành danh sách list
    feature = anime["cleaned_NameGenres"].tolist()
    
    # if để kiểm tra tên anime nhập vào có tồn tại trong bảng hay không. Nếu không, biến "new_title" thành True và thêm tên anime đó vào danh sách feature
    if not(title in feature):
        new_title = True
        feature.append(title)
        
    # Tạo một đối tượng "CountVectorizer" để biểu diễn các chuỗi tiêu đề thành ma trận đếm
    tfidf = text.CountVectorizer()
    
    # Biến đổi danh sách feature thành ma trận đếm sử dụng "fit_transform" của đối tượng CountVectorizer
    tfidf_matrix = tfidf.fit_transform(feature)
    
    # Tính ma trận độ tương đồng cosine từ ma trận đếm "tfidf_matrix"
    similarity = cosine_similarity(tfidf_matrix)
    
    del tfidf
    del tfidf_matrix
    
    # Kiểm tra tên phim vừa nhập có tồn tại hay không
    if (new_title):
        del feature
        # Nếu không tồn tại, tên anime này sẽ nằm cuối của ma trận tương đồng "similarity". Do đó trả về hàng cuối cùng của similarity
        return similarity[len(similarity) - 1]
    
    # Nếu tên phim vừa nhập có tồn tại trong bảng, ta tìm chỉ số "indices" của tiêu đề đó
    else:
        del feature
        index = pd.Series(indices[title])   
        # print(index[0])

        # Trả về hàng tương ứng với chỉ số "index[0]" trong ma trận tương đồng "similarity". Đây là độ tương đồng giữa tên anime nhập vào và các anime khác trong bảng
        return similarity[index[0]]

In [ ]:
# Xây dựng hàm Anime_Recommendation() để gợi ý các anime dựa trên tên và thể loại mà người dùng nhập vào
def Anime_Recommendation():
    
    # Nhập vào từ bàn phím
    name = input("Nhập tên Anime (có thể trống): ")
    genres = input("Nhập thể loại phim (có thể trống): ")
    
    # Hàm clean() được sử dụng để sửa văn bản nhập vào. Kết quả gán cho biến "cleaned_name" và "cleaned_genres"
    cleaned_name = clean(name)
    cleaned_genres = clean(genres)
    
    # Tạo chuỗi kết hợp của tên anime và thể loại đã được chỉnh sửa, cách nhau bởi dấu " "
    cleaned_NameGenres = cleaned_name + " " + cleaned_genres
    
    # Gọi hàm similarity để tính độ tương đồng giữa chuỗi cleaned_NameGenres vừa nhập và các chuỗi khác trong bảng
    similarity = get_similarity(cleaned_NameGenres)
    
    # Biến similarity_scores là một danh sách chứa chỉ số và độ tương đồng từ enumerate(similarity)
    similarity_scores = list(enumerate(similarity))
    
    # Sắp xếp theo độ tương đồng giảm dần sử dụng "sorted"
    # Truyền hàm "lambda" để chỉ định giá trị khóa là phần tử thứ 2 của mỗi cặp chỉ số và độ tương đồng
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    
    # Giữ lại những cặp có độ tương đồng lớn hơn 0, sử dụng hàm filter và hàm lambda để kiểm tra điều kiện
    similarity_scores = filter(lambda x: x[1] > 0, similarity_scores)
    similarity_scores = list(similarity_scores)
    
    # Chọn chỉ số và độ tương đồng của các anime có độ tương đồng lớn hơn 0
    # Biến "movieindices" là một danh sách chứa chỉ số của anime trong "similarity_scores" mà có chỉ số nhỏ hơn độ dài của bảng anime.csv
    movieindices = [i[0] for i in similarity_scores if i[0] < len(anime)]
    
    # Biến "scores" là danh sách chứa độ tương đồng của các anime tương ứng
    scores = [i[1] for i in similarity_scores if i[0] < len(anime)]
    
    # Tạo một DataFrame có tên "result" bằng cách lặp qua các chỉ số trong "movieindices" và trích xuất hàng tương ứng trong bảng anime.csv
    result = pd.DataFrame([anime.iloc[i] for i in movieindices])
    
    # Gán cột "score" cho DataFrame "result" với các giá trị từ danh sách "scores"
    result['score'] = scores
    
    # Chọn các cột 'Name', 'Genres', 'Type', 'Episodes','score' và chuyển đổi thành danh sách list
    # result = result[['Name', 'Genres', 'Type', 'Episodes','score']].values.tolist()
    result = result['Name'].values.tolist()
    
    
    # In thông tin gợi ý Anime
    print("Anime Recommendation: ", name)
    print("Tên phim:", name)
    print("Thể loại phim:", genres) 
    print("Các Anime gợi ý:")
    print(" ")
    
    # In danh sách 10 anime tương đồng bao gồm Tên phim, thể loại, thể loại công chiếu, số tập phim, điểm số tương đồng
    if (len(result) > 10):
        for i in range(0,10):
            print(result[i])
    else:
        for i in range(0, len(result)):
            print(result[i])
            
    del similarity
    del similarity_scores
    del movieindices
    del scores
    del result

In [ ]:
# Chạy hàm Anime_Recommendation()
Anime_Recommendation()